In [ ]:
import preamble
from model import DualAutoencoderModel
from sentence_mapper import SentenceMapper
from globals import FORCE_CURVE_DURATION, FORCE_SAMPLE_COUNT
import pickle
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [ ]:
def derivative_smooth(curve: NDArray) -> NDArray:
    curve = savgol_filter(curve, window_length=50, polyorder=3, axis=0, mode='nearest')
    return np.gradient(curve, FORCE_CURVE_DURATION / FORCE_SAMPLE_COUNT, axis=0)

In [ ]:
modifiers = [
    '',
    'slightly',
    'greatly',
    'smoothly',
    'sharply',
    'slowly',
    'quickly',
    'lightly',
    'significantly',
    'softly',
    'harshly',
    'gradually',
    'immediately',
]

directions = [
    'backward',
    'backward and down',
    'backward and left',
    'backward and right',
    'backward and up',
    'down',
    'down and forward',
    'down and left',
    'down and right',
    'forward',
    'forward and left',
    'forward and right',
    'forward and up',
    'left',
    'left and up',
    'right',
    'right and up',
    'up',
]

# vocabulary = [(f'Move {modifier} {direction}.', np.array([modifier, direction.replace('and ', '')], dtype='U16'))
#               for modifier in modifiers for direction in directions]
# vocabulary += [('', np.array(['', ''], dtype='U16'))]
vocabulary = [(f'Move {modifier} {direction}.', np.array([modifier, direction.split(' ')[0], direction.split(' ')[-1] if 'and' in direction else ''], dtype='U16'))
              for modifier in modifiers for direction in directions]
vocabulary += [('', np.array(['', '', ''], dtype='U16'))]
print(vocabulary)

In [ ]:
# for phrase in [f'{modifier} {direction}' for modifier in modifiers for direction in directions]:
#     print(phrase)

In [ ]:
with open('../models/dae_g.pkl', 'rb') as file:
    dae_b: DualAutoencoderModel = pickle.load(file)

sentence_mapper = SentenceMapper(vocabulary)

In [ ]:
sentence_mapper.map(['GO Swiftly right.'])

In [ ]:
phrases = ["Let's go upward and to the right.", 'I want you to ease ahead over time.', 'I like apples.']
phrase_mappings = sentence_mapper.map(phrases)
print(phrase_mappings)
force_curves = dae_b.phrase_to_force(phrase_mappings)
reconstructed_phrases = dae_b.force_to_phrase(force_curves)

for force_curve, phrase in zip(force_curves, phrases):
    force_curve = derivative_smooth(force_curve)
    time = np.linspace(0.0, FORCE_CURVE_DURATION, FORCE_SAMPLE_COUNT)
    plt.figure(figsize=(5, 2.5))
    plt.plot(time, force_curve[:, 0], color='#DF2935', label='x (right)')
    plt.plot(time, force_curve[:, 1], color='#4DA167', label='y (forward)')
    plt.plot(time, force_curve[:, 2], color='#3C91E6', label='z (up)')
    plt.title(f"\"{phrase}\"", fontweight='bold')
    plt.xlabel('Time (s)')
    plt.ylabel('Force (N)')
    plt.grid(visible=True, alpha=0.25)
    plt.legend()
    plt.ylim(-1.0, 6.0)
    plt.show()